In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

# Downloading IMDb's Top 250 movie data
url = 'https://www.imdb.com/chart/top/?sort=ir,desc&mode=simple&page=1'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

movies = soup.select('td.titleColumn')
links = [a.attrs.get('href') for a in soup.select('td.titleColumn a')]
crew = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]
ratings = [b.attrs.get('data-value') for b in soup.select('td.posterColumn span[name=ir]')]
votes = [b.attrs.get('data-value') for b in soup.select('td.ratingColumn strong')]

imdb = []

# Storing each item into dictionary (data), then put those into a list (IMDb)
for index in range(0, len(movies)):
    # Separate movie into: 'rank', 'title', 'year'
    movie_string = movies[index].get_text()
    movie = (' '.join(movie_string.split()).replace('.', ''))
    movie_title = movie[len(str(index))+1:-7]
    year = re.search('\((.*?)\)', movie_string).group(1)
    rank = movie[:len(str(index))-(len(movie))]
    data = {"movie_title": movie_title,
            "year": year,
            "rank": rank,
            "star_cast": crew[index],
            "rating": ratings[index],
            "vote": votes[index],
            "link": links[index]}
    imdb.append(data)

for item in imdb:
    print(item['rank'], ',', item['movie_title'], '('+item['year']+') ,', item['rating'],',', 'Starring:', item['star_cast'])

df = pd.DataFrame(imdb, columns= ['rank', 'movie_title', 'year', 'rating', 'star_cast'])
df.to_csv (r'C:\Users\(user_name)\Desktop\imdb_rank.csv', index = False, header=True)
print (df)